In [46]:
# Initial imports
import os
import pandas as pd
import requests
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

%matplotlib inline
from dotenv import load_dotenv
load_dotenv()

True

In [47]:
# Reading the News API key enviroment variable
api_key = os.getenv("crypto_news_key")

In [48]:
print(f"API key:{type(api_key)}")

API key:<class 'str'>


In [91]:
crypto_ticker_news = 'https://cryptonews-api.com/api/v1?tickers=ETH&items=50&date=yeartodate&sortby=oldestfirst&page=39&token='+ api_key

In [92]:
response_ticker_news = requests.get(crypto_ticker_news)

In [93]:
response_ticker_news

<Response [200]>

In [94]:
crypto_ticker_news_raw = response_ticker_news.json()

In [95]:
ticker_news_df = pd.DataFrame(data=crypto_ticker_news_raw['data'])

In [96]:
ticker_news_df.tail()

,news_url,image_url,title,text,source_name,date,topics,sentiment,type,tickers
27,https://u.today/associated-press-to-record-bas...,https://crypto.snapi.dev/images/v1/4/s/adobest...,Associated Press to Record Basketball Scores o...,AP has partnered with Everipedia to publish Ma...,UToday,"Thu, 18 Mar 2021 14:56:00 -0400",[],Positive,Article,[ETH]
28,https://cryptoticker.io/en/volmex-finance-defi/,https://crypto.snapi.dev/images/v1/u/4/thunder...,Volmex Finance – Ethereum Makes Possible Token...,It appears that there's no limit to the things...,CryptoTicker,"Thu, 18 Mar 2021 16:59:07 -0400",[],Neutral,Article,[ETH]
29,https://cryptobriefing.com/ethereum-could-redu...,https://crypto.snapi.dev/images/v1/v/2/ethfees...,Ethereum Could Reduce Fees With Batch Transact...,"A new Ethereum proposal, EIP-3074, could reduc...",Crypto Briefing,"Thu, 18 Mar 2021 18:16:05 -0400",[],Neutral,Article,[ETH]
30,https://cryptodaily.co.uk/2021/03/eth-usd-decl...,https://crypto.snapi.dev/images/v1/h/k/drew-be...,ETH/USD Declines Below 1735 After Given at 185...,Ethereum (ETH/USD) was driven lower early in t...,Crypto Daily,"Thu, 18 Mar 2021 20:45:00 -0400",[tanalysis],Negative,Article,[ETH]
31,https://invezz.com/news/2021/03/19/chinese-tec...,https://crypto.snapi.dev/images/v1/h/8/meito-b...,Chinese tech firm Meitu invests an additional ...,"Meitu, a China-based software development comp...",Invezz,"Thu, 18 Mar 2021 21:13:22 -0400",[],Positive,Article,"[BTC, ETH]"


In [97]:
ticker_recent_news_df = ticker_news_df.drop(columns = ['news_url','image_url','source_name',])

In [98]:
ticker_recent_news_df.head()

,title,text,date,topics,sentiment,type,tickers
0,"BTC, ETH, XRP, ADA, EOS, XTZ, AVAX: Technical ...",Ethereum (ETH) was temporarily rejected below ...,"Wed, 17 Mar 2021 07:15:00 -0400",[tanalysis],Neutral,Article,"[ADA, AVAX, BTC, EOS, ETH, XRP, XTZ]"
1,"Bitcoin and Ethereum Struggle, ADA and FIL Out...",Bitcoin price failed to continue higher above ...,"Wed, 17 Mar 2021 08:47:00 -0400",[],Neutral,Article,"[ADA, BTC, ETH]"
2,Binance Smart Chain (BSC) Records New ATH on S...,"Binance Smart Chain (BSC), the native blockcha...","Wed, 17 Mar 2021 08:57:38 -0400",[],Neutral,Article,"[BNB, ETH]"
3,“NFT Tax Shock” Awaits Those Who Pay with Bitc...,CNBC has revealed that those are now buying NF...,"Wed, 17 Mar 2021 09:12:11 -0400",[taxes],Negative,Article,"[BTC, ETH]"
4,Crypto Trader Tyler Swope Predicts Ethereum Wi...,Popular crypto trader and analyst Tyler Swope ...,"Wed, 17 Mar 2021 09:35:20 -0400",[],Neutral,Article,"[BTC, ETH]"


In [96]:
ticker_recent_news_df.sentiment.describe()

count           50
unique           3
top       Positive
freq            35
Name: sentiment, dtype: object

In [97]:
# Create the ETH sentiment scores DataFrame
sentiments = []

for data in crypto_ticker_news_raw['data']:
    try:
        date = data['date']
        text = data["text"]
        results = analyzer.polarity_scores(text)
        compound = results["compound"]
        tickers = data['tickers']
        pos = results["pos"]
        neu = results["neu"]
        neg = results["neg"]

        sentiments.append({
            "date": date,
            "text": text,
            "tickers":tickers,
            "Compound": compound,
            "Positive": pos,
            "Negative": neg,
            "Neutral": neu,
        })
    except AttributeError:
        pass
    
events  = pd.DataFrame(sentiments)

In [99]:
events

,date,text,tickers,Compound,Positive,Negative,Neutral
0,"Tue, 01 Dec 2020 10:46:38 -0500","When Bitcoin's genesis block first launched, a...",[ETH],-0.2263,0.077,0.075,0.848
1,"Tue, 01 Dec 2020 12:21:53 -0500",Ethereum is moving from a proof-of-work blockc...,[ETH],0.6486,0.128,0.000,0.872
2,"Tue, 01 Dec 2020 18:18:17 -0500","Ethereum 2.0's beacon chain goes live, heraldi...","[BTC, ETH]",0.4215,0.093,0.039,0.869
3,"Wed, 02 Dec 2020 08:26:59 -0500","Powered by Every week, IntoTheBlock brings you...","[BTC, ETH]",0.2023,0.053,0.000,0.947
4,"Thu, 03 Dec 2020 09:31:21 -0500",The US crypto exchange is matching donations r...,[ETH],0.0000,0.000,0.000,1.000
5,"Thu, 03 Dec 2020 10:00:18 -0500","StrongBlock, a protocol designed to reward and...",[ETH],0.7269,0.124,0.000,0.876
6,"Thu, 03 Dec 2020 12:44:06 -0500",Leading decentralized finance protocol Aave ha...,"[AAVE, ETH]",0.2023,0.049,0.000,0.951
7,"Thu, 03 Dec 2020 17:58:20 -0500",The IRS has yet to provide specific guidance o...,[ETH],0.0000,0.000,0.000,1.000
8,"Fri, 04 Dec 2020 04:43:02 -0500",Over $600 million is now staked on the world's...,[ETH],0.1280,0.081,0.000,0.919
9,"Fri, 04 Dec 2020 06:29:09 -0500","Powered by Every week, IntoTheBlock brings you...","[BTC, ETH]",0.2023,0.053,0.000,0.947
